import everything

In [ ]:
import sqlite3
import os
import re
from pprint import pprint
# import dateutil

Create a new database connection cursor

In [22]:
con = sqlite3.connect('/home/klb807/git/Work_Tools/ASAP_sqlite/iMCI_P60induction_data.sqlite')
cur = con.cursor()
# res = cur.execute('SELECT * FROM MOUSE;')
# if res.fetchone() is None:
#     print('Empty Table')

# Create the tables


---------------------------------

## *Mouse*
List of all mice recorded

| Field | Datatype | Comments |
| ---- | ---- | ---- |
| id | text | combo of cage ID and ear tag |
| injection date | text | YYYY-mm-ddTHH:MM |
| injection type | text | No injection, THs-Cre, or just marker |
| sex | text | self explanatory |



---------------------------------

## *Sessions*
List of recording sessions. Typically three per day

| Field | Datatype | Comments |
| ---- | ---- | ---- |
| mouse_id | text | foreign key from mouse table |
| time | text | recording time |
| task | text | chochip, openfield, sticker |
| experimenter | text | usually Arin |
| enclosure | text | openfield, new box, old box |
| comments | text | anything to add ?|



---------------------------------

## *Videos*
List of videos and their locations. Not just original recordings, but also 
includes the cropped and edited videos. 

| Field | Datatype | Comments |
| ---- | ---- | ---- |
| relative_path | text | path from the project base (iMCP_p60 etc) |
| session_id | text | foreign key from Sessions table |
| description | text | is this the original recording? A converted mp4? Cropped? |


---------------------------------

## *Calibrations*
Intrinsic and Extrinsic matrices for multi-view, plus the bounding boxes for each view

| Field | Datatype | Comments |
| ---- | ---- | ---- |
| relative_path | text | where is the recording for the associated calibration? |
| boundary | blob | encoded dictionary of the boundary points for each view |
| intrinsic | blob | encoded dictionary of camera calibration matrices |
| extrinsic | blob | encoded dictionary of camera calibration matrices |



In [ ]:
# create the mouse table
mouse_creation = '''
                    CREATE TABLE mouse 
                    (id text, injection_date text, injection_type text, sex text);
                    '''

cur.execute(mouse_creation)

# create the recordings table
session_creation = ''' 
                    CREATE TABLE session
                    (mouse_id text, time text, task text, experimenter text,
                    enclosure text, comments text,
                    FOREIGN KEY (mouse_id) REFERENCES "mouse" ([id]));
                    '''
cur.execute(session_creation)


# create the videos table
videos_creation = ''' 
                    CREATE TABLE videos (
                        relative_path text, 
                        session_id text,
                        description text,
                        FOREIGN KEY (session_id) REFERENCES "session" ([rowid])
                    );'''

cur.execute(videos_creation)


# calibration table
calibration_creation = ''' 
                    CREATE TABLE calibration (
                        relative_path text,
                        boundary blob,
                        intrinsic blob,
                        extrinsic blob
                    );'''
cur.execute(calibration_creation)

Let's make sure that everything was created

In [ ]:
# cur.execute('.headers on; .mode columns;')

cur.execute("PRAGMA table_info('mouse')")
pprint(cur.fetchall())
cur.execute("PRAGMA table_info('session')")
pprint(cur.fetchall())
cur.execute("PRAGMA table_info('videos')")
pprint(cur.fetchall())
cur.execute("PRAGMA table_info('calibration')")
pprint(cur.fetchall())

# Insert the p60 mice into the tables

In [ ]:
mouse_insert ='''   INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596507_5674', 'M', '07/12/2023', 'no injection');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596507_5675', 'M', '07/12/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596507_5676', 'M', '07/11/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596507_5682', 'M', '07/11/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596507_5683', 'M', '07/11/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596506_5677', 'F', '07/12/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596506_5678', 'F', '07/12/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596506_5679', 'F', '07/11/2023', 'no injection');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596506_5680', 'F', '07/12/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2596506_5681', 'F', '07/11/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605920_6285', 'M', '08/10/2023', 'no injection');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605920_6286', 'M', '08/10/2023', 'AV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605920_6287', 'M', '08/09/2023', 'AV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605920_6288', 'M', '08/10/2023', 'AV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605920_6290', 'M', '08/09/2023', 'AV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605373_6227', 'F', '08/08/2023', 'AV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605373_6228', 'F', '08/10/2023', 'AV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605373_6229', 'F', '08/08/2023', 'AV9-THs-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605373_6230', 'F', '08/08/2023', '9-THs-Cre-2A-FusionRed-mWPRE-hGHpA');
                    INSERT INTO mouse (id, sex, injection_date, injection_type) VALUES ('2605373_6231', 'F', '08/10/2023', 'no injection');'''


cur.executescript(mouse_insert)

In [16]:
# and make sure that it actually stuck
cur.execute('SELECT * FROM mouse;')
pprint(cur.fetchall())

[('2596507_5674', '07/12/2023', 'no injection', 'M'),
 ('2596507_5675', '07/12/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA', 'M'),
 ('2596507_5676', '07/11/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ', 'M'),
 ('2596507_5682', '07/11/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA', 'M'),
 ('2596507_5683', '07/11/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ', 'M'),
 ('2596506_5677', '07/12/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ', 'F'),
 ('2596506_5678', '07/12/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA', 'F'),
 ('2596506_5679', '07/11/2023', 'no injection', 'F'),
 ('2596506_5680', '07/12/2023', 'AAV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ', 'F'),
 ('2596506_5681', '07/11/2023', 'AAV9-THs-FusionRed-mWPRE-hGHpA', 'F'),
 ('2605920_6285', '08/10/2023', 'no injection', 'M'),
 ('2605920_6286', '08/10/2023', 'AV9-THs-Cre-2A-FusionRed-mWPRE-hGHpA ', 'M'),
 ('2605920_6287', '08/09/2023', 'AV9-THs-FusionRed-mWPRE-hGHpA', 'M'),
 ('2605920_6288', '08/10/2023', 'AV9-THs-FusionRed-mWPRE-hGHpA', 'M'),
 ('260592

Iterate through the directories to get the list of recordings, tiffs and the videos

In [17]:
for root,dirs,files in os.walk('/mnt/Kennedy_SMB/ASAP/iMCI-P60induction/', topdown=False):
    
    # skip if there aren't any .tiff or .mp4
    match_files = [file for file in files if '.tiff' in file or '.mp4' in file] 

    # skip if there aren't any tiffs or mp4s
    if not match_files:
        continue

    # parsing the mouse id 
    mouse_id = re.search('(\d{7}_\d{4})', root).group(0)
    
    # parsing the recording date
    rec_date = re.search('(2023\d{4})_(\d{9})', match_files[0])
    parsed_recdate = f'{rec_date.group(1)[0:4]}-{rec_date.group(1)[4:6]}-{rec_date.group(1)[6:]}'
    parsed_rectime = f'{rec_date.group(2)[0:2]}:{rec_date.group(2)[2:4]}:{rec_date.group(2)[4:6]}'
    parsed_dt = parsed_recdate+'T'+parsed_rectime
    
    # parsing the task
    task_id = re.search('(chochip|openfield|sticker)', root).group(0)

    # enclosure type
    enclosure = 'openfield' if task_id == 'openfield' else 'small_multiview'
    
    # session insert
    rec_query = f'''INSERT INTO session (mouse_id, time, task, enclosure)
        VALUES ('{mouse_id}', '{parsed_dt}', '{task_id}', '{enclosure}');'''
    
    # print(rec_query)
    cur.execute(rec_query)
    rec_id = cur.lastrowid()

    # insert vid

    # mp4 file list
    mp4_files = [os.path.join(root, file) for file in match_files if '.mp4' in file]

    # sql query
    for file in mp4_files:
        cur.execute(f'''INSERT INTO videos (relative_path, session_id) VALUES ('{file}', '{rec_id}')''')
        vid_id = cur.fetchone()

    # print if we got it
    if vid_id and rec_id:
        print('Inserted session and videos')



In [25]:
con.close()

In [23]:
cur.execute('SELECT COUNT(*) FROM session;')

In [24]:
cur.fetchall()

[(712,)]

In [ ]:
cur.execute('DELETE FROM session;')


In [20]:
con.commit()